In [7]:
# Backprop on the Seeds Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as mtp
from math import*
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.model_selection import train_test_split

In [8]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

In [9]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

In [10]:
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

In [11]:
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

In [12]:
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [13]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

In [14]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

In [15]:
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))

In [16]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

In [17]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

In [18]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [19]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

In [20]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [36]:
# dataset=[[0,0,1],[0,1,0],[1,0,0],[1,1,1]]
dataset1 = load_csv('/content/banknote.csv')
for i in range(len(dataset1[0])-1):
	str_column_to_float(dataset1, i)
# convert class column to integers
str_column_to_int(dataset1, len(dataset1[0])-1)
# normalize input variables
minmax = dataset_minmax(dataset1)
normalize_dataset(dataset1, minmax)

In [39]:
dataset, testing_data = train_test_split(dataset1, test_size=0.3)
# print(len(df),len(testing_data))

In [40]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.7, 1000, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.700, error=486.957
>epoch=1, lrate=0.700, error=277.017
>epoch=2, lrate=0.700, error=119.954
>epoch=3, lrate=0.700, error=67.970
>epoch=4, lrate=0.700, error=53.023
>epoch=5, lrate=0.700, error=47.580
>epoch=6, lrate=0.700, error=44.738
>epoch=7, lrate=0.700, error=42.735
>epoch=8, lrate=0.700, error=41.193
>epoch=9, lrate=0.700, error=39.967
>epoch=10, lrate=0.700, error=39.070
>epoch=11, lrate=0.700, error=38.418
>epoch=12, lrate=0.700, error=37.886
>epoch=13, lrate=0.700, error=37.418
>epoch=14, lrate=0.700, error=36.995
>epoch=15, lrate=0.700, error=36.614
>epoch=16, lrate=0.700, error=36.286
>epoch=17, lrate=0.700, error=36.001
>epoch=18, lrate=0.700, error=35.726
>epoch=19, lrate=0.700, error=35.441
>epoch=20, lrate=0.700, error=35.145
>epoch=21, lrate=0.700, error=34.846
>epoch=22, lrate=0.700, error=34.548
>epoch=23, lrate=0.700, error=34.244
>epoch=24, lrate=0.700, error=33.928
>epoch=25, lrate=0.700, error=33.603
>epoch=26, lrate=0.700, error=33.280
>epoch=2

In [24]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

In [41]:
c=0
for row in testing_data:
  prediction = predict(network, row)
  if row[-1] == prediction:
    c+=1
  print('Expected=%d, Got=%d' % (row[-1], prediction))
	

Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=0

In [42]:
pre=[predict(network,row) for row in testing_data]
print('Accuracy is :',(c/len(pre))*100)

Accuracy is : 100.0
